In [1]:
#Import necessary packages, load the point cloud

import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os

pcd = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "pcd.ply"))
num = len(pcd.points)
print("Number of points before downsampling: ", num)



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Number of points before downsampling:  18914399


In [2]:
#Downsample the pointcloud to make it easier to work with

pcd_downsampled = pcd.voxel_down_sample(voxel_size=0.05)
print("Number of points after downsampling: ", len(pcd_downsampled.points))
print("Reduced the amount of points by : ", (num - len(pcd_downsampled.points))/num * 100, "%")

Number of points after downsampling:  7539537
Reduced the amount of points by :  60.13863829350328 %


In [3]:
#Vizualize the downsampled pointcloud
o3d.visualization.draw_geometries([pcd_downsampled])

In [4]:
#Statistical outlier removal

pcd_filtered, ind = pcd_downsampled.remove_statistical_outlier(nb_neighbors=20, std_ratio=2)
print("Removed ", len(pcd_downsampled.points) - len(pcd_filtered.points), " points")

Removed  196966  points


In [5]:
#Vizualise the filtered pointcloud
o3d.visualization.draw_geometries([pcd_filtered])

In [15]:
#Slicing the point cloud through the tree trunks based on the z-value, between 1 and 2, 2 and 3, 3 and 4


pcd_filtered_array = np.asarray(pcd_filtered.points)
pcd_filtered_array = pcd_filtered_array[np.where(pcd_filtered_array[:,2] > 1.0)]
pcd_filtered_array = pcd_filtered_array[np.where(pcd_filtered_array[:,2] < 2.0)]

pcd_filtered_array_2 = np.asarray(pcd_filtered.points)
pcd_filtered_array_2 = pcd_filtered_array[np.where(pcd_filtered_array[:,2] > 2.0)]
pcd_filtered_array_2 = pcd_filtered_array[np.where(pcd_filtered_array[:,2] < 3.0)]

pcd_filtered_array_3 = np.asarray(pcd_filtered.points)
pcd_filtered_array_3 = pcd_filtered_array[np.where(pcd_filtered_array[:,2] > 3.0)]
pcd_filtered_array_3 = pcd_filtered_array[np.where(pcd_filtered_array[:,2] < 4.0)]


pcd_sliced_1 = pcd_filtered.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(-100, -100, 0.2), max_bound=(100, 100, 0.4)))
pcd_sliced_2 = pcd_filtered.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(-100, -100, 0.2), max_bound=(100, 100, 0.4)))
pcd_sliced_3 = pcd_filtered.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(-100, -100, 0.2), max_bound=(100, 100, 0.4)))





In [16]:
o3d.visualization.draw_geometries([pcd_sliced_1])
o3d.visualization.draw_geometries([pcd_sliced_2])
o3d.visualization.draw_geometries([pcd_sliced_3])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 


In [8]:
#Cluster the sliced cloud so we can run ransac on each cluster

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd_sliced.cluster_dbscan(eps=0.6, min_points=30, print_progress=True)) 

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 204
point cloud has 204 clusters


In [9]:
individual_point_clouds = []

for cluster_label in range(max_label + 1):
    
    cluster_indices = np.where(labels == cluster_label)[0]

    cluster_points = pcd_sliced.select_by_index(cluster_indices)

    individual_point_clouds.append(cluster_points)

    del(cluster_points)
    del(cluster_indices)

In [ ]:
#Run ransac on each cluster to find the best cylinder
import pyransac3d as pyrsc

cluster_1 = individual_point_clouds[0]

model_cylinder = pyrsc.Cylinder()

center, axis, radius, inliers = model_cylinder.fit(np.asarray(cluster_1.points), thresh=0.25, maxIteration = 1000)

tree_trunk = cluster_1.select_by_index(inliers)

o3d.visualization.draw_geometries([tree_trunk]) 
    


In [10]:
for pcd in individual_point_clouds:
    o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 


KeyboardInterrupt: 

In [11]:
#Running ransac on each cluster

import pyransac3d as pyrsc

circle_cloud = o3d.geometry.PointCloud()

#Define the model
model_cylinder = pyrsc.Cylinder()


for pcd in individual_point_clouds:
    
    #Loading the points as per pyransac3d documentation
    load_points = np.asarray(pcd.points) 
    
    #Fitting the circle model to the points
    center, axis, radius, inliers = model_cylinder.fit(load_points, thresh=0.05, maxIteration = 200) 
    
    #Selecting the inliers
    inlier_cloud = pcd.select_by_index(inliers) 
    
    #Load inlier cloud points and colors
    load_inlier_points = np.asarray(inlier_cloud.points)
    load_inlier_colors = np.asarray(inlier_cloud.colors)
    
    #Load circle_cloud points and colors
    load_circle_points = np.asarray(circle_cloud.points)
    load_circle_colors = np.asarray(circle_cloud.colors)
    
    #Make a new set of points and colors
    new_points = np.concatenate((load_inlier_points, load_circle_points), axis = 0)
    new_colors = np.concatenate((load_inlier_colors, load_circle_colors), axis = 0)
    
    #Append the new points to cloud
    circle_cloud.points = o3d.utility.Vector3dVector(new_points)
    circle_cloud.colors = o3d.utility.Vector3dVector(new_colors)
    
    
    
    
    

In [14]:
circle_cloud.paint_uniform_color([0, 0, 1])
o3d.visualization.draw_geometries([pcd_sliced])
o3d.visualization.draw_geometries([circle_cloud, pcd_sliced])

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
